<a href="https://colab.research.google.com/github/JayDown/3d-converter/blob/master/MediaPipe_Iris_model_converter_for_Barracuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MediaPipe Iris model converter for Unity Barracuda

## What it does

- Converts .tflite into ONNX using tflite2onnx
- Replace Pad operators with combinations of ConstantOfShape and Concat.
- Add Expand operators to PRelu slope inputs.

## Why it's needed

- The current implementation of Barracuda doesn't support non-spatial axis padding, so I had to replace them with concatenation with zero-filled tensors.
- The current implementation of the PRelu activator in Barracuda doesn't support undirectional broadcasting, so I had to expand the slope tensors before feeding to the activators.

# Setup

In [ ]:
%pip install tflite2onnx

     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 14.5MB 303kB/s 
     |████████████████████████████████| 92kB 7.9MB/s 


# tflite to ONNX conversion

In [ ]:
!wget https://github.com/google/mediapipe/raw/master/mediapipe/models/iris_landmark.tflite

--2021-04-14 04:27:55--  https://github.com/google/mediapipe/raw/master/mediapipe/models/iris_landmark.tflite
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/iris_landmark.tflite [following]
--2021-04-14 04:27:56--  https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/iris_landmark.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2640568 (2.5M) [application/octet-stream]
Saving to: ‘iris_landmark.tflite’

iris_landmark.tflit 100%[===================>]   2.52M  --.-KB/s    in 0.1s    

2021-04-14 04:27:57 (18.9 MB/s) - ‘iris_lan

In [ ]:
!tflite2onnx iris_landmark.tflite iris_landmark.onnx

# Converter implementation

In [ ]:
import numpy as np
import onnx
from onnx import checker, helper
from onnx import AttributeProto, TensorProto, GraphProto
from onnx import numpy_helper as np_helper

In [ ]:
# Shape tensor generator
def get_shape_tensor(model, shape):
  name = 'shape_{0}x{1}x{2}x{3}'.format(*shape)

  # If the initializer already exists, simply use it.
  exists = any(x for x in model.graph.initializer if x.name == name)
  if exists: return name

  # Add the initializer for the tensor.
  tensor = helper.make_tensor(name, TensorProto.INT64, (4,), shape)
  model.graph.initializer.append(tensor)
  return name

## Pad operator replacement

In [ ]:
def replace_pad_ops(model):
  i = 0
  while i < len(model.graph.node):
    # Node type check
    node = model.graph.node[i]
    if node.op_type != 'Pad': i += 1; continue

    # Pad node input
    data = next(n for n in model.graph.value_info  if n.name == node.input[0])
    pads = next(n for n in model.graph.initializer if n.name == node.input[1])

    # Shape tensor
    dim = tuple(map(lambda x: x.dim_value, data.type.tensor_type.shape.dim))
    ext = np_helper.to_array(pads)[5]
    shape_tensor = get_shape_tensor(model, (1, ext, dim[2], dim[3]))

    # Replacement nodes
    const_out = node.name + '_pad'
    const_node = helper.make_node('ConstantOfShape', (shape_tensor,), (const_out,))
    concat_node = helper.make_node('Concat', (data.name, const_out), (node.output[0],), axis = 1)

    # Graph modification
    model.graph.node.insert(i, const_node)
    model.graph.node.insert(i + 1, concat_node)
    model.graph.node.remove(node)
    i += 2

## PRelu operator replacement

In [ ]:
def replace_prelu_ops(model):
  i = 0
  while i < len(model.graph.node):
    # Node type check
    node = model.graph.node[i]
    if node.op_type != 'PRelu': i += 1; continue

    # PRelu node input
    input = next(n for n in model.graph.value_info if n.name == node.input[0])

    # Shape tensor
    dim = tuple(map(lambda x: x.dim_value, input.type.tensor_type.shape.dim))
    shape_tensor = get_shape_tensor(model, dim)

    # Replacement nodes
    expand_out = node.name + '_expand'
    expand_node = helper.make_node('Expand', (node.input[1], shape_tensor), (expand_out,))
    prelu_node = helper.make_node('PRelu', (input.name, expand_out), (node.output[0],))

    # Graph modification
    model.graph.node.insert(i, expand_node)
    model.graph.node.insert(i + 1, prelu_node)
    model.graph.node.remove(node)
    i += 2

# ONNX to ONNX (Barracuda) conversion

In [ ]:
model = onnx.load("iris_landmark.onnx")
replace_pad_ops(model)
replace_prelu_ops(model)
checker.check_model(model)
onnx.save(model, "iris_landmark_barracuda.onnx")